# Mushroom Classification With Artificial Neural Networks

## Introduction
In this notebook, we use neural networks in an attempt to classify mushrooms as poisonous or edible based on observed physical characteristics of the mushrooms.

#### Import required libraries

In [31]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.compose import ColumnTransformer 
from sklearn.preprocessing import LabelBinarizer
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.metrics import classification_report, make_scorer, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import StratifiedKFold, KFold, cross_val_score, cross_validate, train_test_split, GridSearchCV
from sklearn.linear_model import Perceptron


In [32]:
# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

The latest Pandas version is required 

In [33]:
# Requires Pandas 1.2.4
#!pip install --upgrade pandas==1.2.4 --user
#pd.__version__

#### Define functions used later

In [34]:
def one_hot_encode(data, fields):
    # One-hot encodes categorical data with n unique values
    # into n new fields
    
    # Inputs: 
    #    data:   Pandas dataframe containing dataset 
    #    fields: Fields to encode
    
    # Returns: Updated dataset
    
    onehot = pd.get_dummies(data[fields])
    data = pd.concat([onehot, data], axis=1)
    data = data.drop(columns=fields, axis=1)
    data = data.reset_index(drop=True)
    return data

 

def update_results(description, metrics, results_table=pd.DataFrame()):
    
    # Updates results in Pandas dataframe
    
    # Inputs: 
    #    description:   Pandas dataframe containing dataset 
    #    metrics:       Field to be binned
    #    results_table: Required number of bins
    
    # Returns: Updated results table
    
    u = pd.DataFrame(metrics).mean(axis=0)
    v = pd.Series({'Description' : description})
    t = pd.DataFrame(v.append(u)).transpose()
    return results_table.append(t)


#### Define globals here

In [35]:
# Globals
# Scoring for kfolds cross validation
scoring = {'accuracy' : make_scorer(accuracy_score), 
           'precision' : make_scorer(precision_score, average = 'weighted'),
           'recall' : make_scorer(recall_score, average = 'weighted'), 
           'f1_score' : make_scorer(f1_score, average = 'weighted')} 

# Set variables to hold results for tests to follow
results = []
names = []

#### Load dataset

In [36]:
mushroom_data = pd.read_csv('newMushroom.csv')

In [37]:
mushroom_data.shape

(8124, 117)

In [38]:
mushroom_data.head()

,class,cap-shape_b,cap-shape_c,cap-shape_f,cap-shape_k,cap-shape_s,cap-shape_x,cap-surface_f,cap-surface_g,cap-surface_s,...,population_s,population_v,population_y,habitat_d,habitat_g,habitat_l,habitat_m,habitat_p,habitat_u,habitat_w
0,1,0,0,0,0,0,1,0,0,1,...,1,0,0,0,0,0,0,0,1,0
1,0,0,0,0,0,0,1,0,0,1,...,0,0,0,0,1,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
3,1,0,0,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
4,0,0,0,0,0,0,1,0,0,1,...,0,0,0,0,1,0,0,0,0,0


#### Correlated features
Check for perfectly correlated feature vectors.

In [39]:
cols = mushroom_data.columns
dupes = []
for i in cols:
    for j in cols:
        if (sum(mushroom_data[i] != mushroom_data[j]) == 0) & (cols.to_list().index(j) > cols.to_list().index(i)):
            dupes.append(j)
dupes = set(dupes)

The following features have been identified as 100% correlated with others and can therefore be removed as they provide no additional information for our models.

In [40]:
dupes

{'ring-number_n',
 'ring-type_n',
 'stalk-color-above-ring_c',
 'stalk-color-below-ring_c',
 'stalk-color-below-ring_o',
 'veil-color_y'}

In [41]:
# Drop correlated columns

mushroom_data = mushroom_data.drop(dupes, axis=1)
mushroom_data.shape

(8124, 111)

In [42]:
# Drop any duplicate rows created by the column removal
mushroom_data = mushroom_data[~mushroom_data.duplicated()]
mushroom_data.shape

(8124, 111)

#### Define dataset columns

In [43]:
# Define column types
target_col = ['class']
categorical_cols = [i for i in mushroom_data.columns if i not in ['class']]

#### Set aside data for final test evaluation

In [44]:
y = mushroom_data[target_col]
X = mushroom_data[categorical_cols]


#Create unseen holdout set for final evaluation
X, X_unseen, y, y_unseen = train_test_split(X, y, test_size=0.1, random_state=19, stratify=y, shuffle=True)

# Reset X and y indices for later merging
y = y.reset_index(drop=True)
X = X.reset_index(drop=True)

#### Benchmark Model
Here we train a benchmark single-layer perceptron model, the results from which will be used to gauge the performance of later model variations. 


In [45]:
# Benchmark MLP model

name = 'Benchmark'
model = MLPClassifier(random_state=14)
kfold = StratifiedKFold(n_splits=5, random_state=15, shuffle=True)
metrics = cross_validate(model, X, y.values.ravel(), cv=kfold, scoring = scoring)

results.append(metrics)
names.append(name)
table = update_results(name, metrics)

In [46]:
table

,Description,fit_time,score_time,test_accuracy,test_precision,test_recall,test_f1_score
0,Benchmark,1.674918,0.007857,1.0,1.0,1.0,1.0


##### Comment
Our benchmark model has, apparently, returned perfect results, correctly classifying 7311 records in our data.  This might initially seem implausible - that a function exists that perfectly separates poisonous from edible varieties. To convince ourselves that this isn't due to some kind of systemic problem or an anomaly with the input data, we rerun the experiment with a single-layer perceptron handicapped by limiting the number of epochs. If our handicapped model makes classification errors, it will give us more confidence that, with sufficient epochs, weights are converging in order to give the perfect result observed above.

In [77]:
# Hancicapped MLP

name = 'Handicapped MLP'
model = MLPClassifier(random_state=14, max_iter=2)
kfold = StratifiedKFold(n_splits=5, random_state=15, shuffle=True)
metrics = cross_validate(model, X, y.values.ravel(), cv=kfold, scoring = scoring)

results.append(metrics)
names.append(name)
table = update_results(name, metrics)
table

,Description,fit_time,score_time,test_accuracy,test_precision,test_recall,test_f1_score
0,Handicapped MLP,0.151434,0.009587,0.979755,0.979962,0.979755,0.979744


##### Comment
Our model trained over just two epochs makes some classification errors. After max_iter is adjusted back to 15 (not shown), the model's weights converge to provide a perfect classifier .  Below we evaluate on unseen test data.

In [78]:
# Test on unseen data
model = MLPClassifier(random_state=14)
y_pred = model.fit(X,y).predict(X_unseen)
print("F1 score: " + str(f1_score(y_unseen, y_pred, average='weighted',zero_division=0).round(3)))
print("Accuracy: " + str(accuracy_score(y_unseen, y_pred).round(3)))

F1 score: 1.0
Accuracy: 1.0


Again, we observe perfect f1 and accuracy scores.

Below, we simplify further to determine if a single Perceptron is capable of similar predictive performance.

In [79]:
# Single Perceptron model

name = 'Single Perceptron'
model = Perceptron(random_state=14)
kfold = StratifiedKFold(n_splits=5, random_state=15, shuffle=True)
metrics = cross_validate(model, X, y.values.ravel(), cv=kfold, scoring = scoring)

results.append(metrics)
names.append(name)
table = update_results(name, metrics)

In [80]:
table

,Description,fit_time,score_time,test_accuracy,test_precision,test_recall,test_f1_score
0,Single Perceptron,0.03041,0.005219,0.999863,0.999863,0.999863,0.999863


#### Results from individual k-folds

In [50]:
metrics

{'fit_time': array([0.03490567, 0.02497244, 0.02091146, 0.02197433, 0.02289772]),
 'score_time': array([0.00698018, 0.00497937, 0.00498652, 0.00396681, 0.00488615]),
 'test_accuracy': array([0.99931647, 1.        , 1.        , 1.        , 1.        ]),
 'test_precision': array([0.99931744, 1.        , 1.        , 1.        , 1.        ]),
 'test_recall': array([0.99931647, 1.        , 1.        , 1.        , 1.        ]),
 'test_f1_score': array([0.99931649, 1.        , 1.        , 1.        , 1.        ])}

##### Comment
Whilst one of the k-fold validations above returned less-than-perfect results, the other four achieved f1 scores of 1. This points to there being a linear combination of the one-hot-encoded inputs that determines if a mushroom is poisonous or edible. Below we test the Perceptron on unseen test data.

In [51]:
# Test on unseen data
model = Perceptron(random_state=14)
y_pred = model.fit(X,y).predict(X_unseen)
f1_score(y_unseen, y_pred, average='micro',zero_division=0)
print("F1 score: " + str(f1_score(y_unseen, y_pred, average='weighted',zero_division=0).round(3)))
print("Accuracy: " + str(accuracy_score(y_unseen, y_pred).round(3)))

F1 score: 1.0
Accuracy: 1.0


Once again, our perceptron model has made no predictive errors, even on data that it has never seen before.

Below are the converged model weights of this trained Perceptron. A linear combination of these and the binary feature input values determines a mushroom's classification. In other words, edible and poisonous mushrooms are linearly separable.

In [52]:
model.coef_

array([[  2.,   1.,   0.,   1.,  -2.,  -1.,  -2.,   2.,   0.,   1.,   4.,
         -4.,   0.,   0.,  -2.,   4.,  -2.,  -1.,   2.,   0.,   0.,   1.,
          1.,   0.,   7.,  -6.,  -8.,   9.,   6.,  -1.,   0.,  -2.,  -2.,
         -2.,   0.,  -2.,   3.,   0.,  -1.,   2., -11.,  11.,  12., -11.,
          2., -13.,   7.,   1.,   3.,   3.,  -2.,  -1.,   9.,  -2.,  -1.,
         -4.,  -3.,   7.,  -4.,   1.,  -5.,   2.,  -2.,   6.,   0.,  -2.,
         -1.,  -2.,  -1.,   2.,  -1.,   4.,   1.,  -1.,   0.,  -5.,   1.,
          0.,   4.,  -1.,   0.,  -2.,  -1.,   0.,   3.,  -6.,   1.,   1.,
         -1.,   4.,  -5.,  -6.,   0.,  13.,  -6.,   2.,   0.,  -1.,   7.,
         -2.,   3.,   1.,  -7.,  -1.,   2.,   2.,   3.,  -4.,   2.,  -3.]])